In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

%store -r api_key
%store -r token
%matplotlib inline

### Using the recent search endpoint 🌎

In [2]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'

### Setting the parameters for text, time tweeted, tweet metrics, date & time, and how many tweets per request.

In [3]:
params = {
    'query':'(of OR to OR be OR have) lang:en -is:retweet -onlyfans -dick -tits -motor -cock -penis -bitch -fuck -vagina -anal -jizz -sex -#jpzbot -#masturbation -#sex -#hottiktokgirls',
    'tweet.fields':'created_at,public_metrics',
    'end_time':'2021-04-19T12:30:00Z',
    'max_results':100
}

In [4]:
response = requests.get(endpoint, params = params, headers = {'Authorization': 'Bearer '+token})

In [5]:
response

<Response [200]>

In [6]:
res = response.json()

In [7]:
res

{'data': [{'text': "@Sportyfreak2005 ISL played an huge role in developing of Football in India &amp; bringing up fan accorss India... But the case in European Super League is completely different ! It's just the e matter of greediness",
   'id': '1384122103977316355',
   'created_at': '2021-04-19T12:29:59.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 1,
    'quote_count': 0}},
  {'text': '@O_ssai @AyoOyalowo Was having a conversation with a Carpenter who was working alone. Asked him why he doesn\'t have any apprentice, he laughed and he was like, kids of nowadays will rather do "yahoo" than gett a real job. It\'s a sad situation.',
   'id': '1384122103918596098',
   'created_at': '2021-04-19T12:29:59.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 1,
    'like_count': 7,
    'quote_count': 0}},
  {'text': 'Money stop nonsense is the main reason why most of those clubs are leaving UEFA to form ESL🥂',
   'id': '1384122103876

### Fetching data 🐿

In [8]:
data = []
def get_data(n_page):
    global params
    if n_page:
        params['next_token'] = n_page
    twitter_res = requests.get(endpoint, params = params, headers = {'Authorization': 'Bearer '+token})
    if twitter_res:
        json = twitter_res.json()
        for item in json['data']:
            data.append(item)
        if len(data) <= 25000:
            n_page = json['meta']['next_token']
            get_data(n_page)
get_data(False)

print(len(data), data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
print(len(data))

25032


### Setting up the dataframe! 😃

In [12]:
twitter_tweets = pd.DataFrame(list(data))

In [13]:
twitter_tweets

,created_at,text,id,public_metrics,withheld
0,2021-04-19T12:29:59.000Z,@Sportyfreak2005 ISL played an huge role in de...,1384122103977316355,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
1,2021-04-19T12:29:59.000Z,@O_ssai @AyoOyalowo Was having a conversation ...,1384122103918596098,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
2,2021-04-19T12:29:59.000Z,Money stop nonsense is the main reason why mos...,1384122103876648967,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN
3,2021-04-19T12:29:59.000Z,Watching PMs speech right now and honestly it'...,1384122103863996422,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",NaN
4,2021-04-19T12:29:59.000Z,@Richard_1942 It's almost as if being part of ...,1384122103822045194,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
...,...,...,...,...,...
25027,2021-04-19T12:27:56.000Z,guys i miss xinyan so much where is she why do...,1384121584185577480,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
25028,2021-04-19T12:27:56.000Z,@Banana_da_boi Where and what kind of,1384121584164622342,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",NaN
25029,2021-04-19T12:27:56.000Z,@schittock7 Probably on the phone to Niko Kran...,1384121584164622336,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
25030,2021-04-19T12:27:56.000Z,@HanBigTiddies TAENA 😭😭✋NO TO MOVE ONN,1384121584160448513,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN


In [14]:
twitter_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25032 entries, 0 to 25031
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   created_at      25032 non-null  object
 1   text            25032 non-null  object
 2   id              25032 non-null  object
 3   public_metrics  25032 non-null  object
 4   withheld        2 non-null      object
dtypes: object(5)
memory usage: 977.9+ KB


**Looking at the value in the public_metrics column for the first row. Planning to unpack the dictionary into separate columns.**

In [16]:
twitter_tweets['public_metrics'][0]

{'retweet_count': 0, 'reply_count': 0, 'like_count': 1, 'quote_count': 0}

In [19]:
twitter_tweets[['retweet_count','reply_count','like_count','quote_count']] = twitter_tweets.public_metrics.apply(pd.Series)
twitter_tweets

,created_at,text,id,public_metrics,withheld,retweet_count,reply_count,like_count,quote_count
0,2021-04-19T12:29:59.000Z,@Sportyfreak2005 ISL played an huge role in de...,1384122103977316355,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0,0,1,0
1,2021-04-19T12:29:59.000Z,@O_ssai @AyoOyalowo Was having a conversation ...,1384122103918596098,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,0,1,7,0
2,2021-04-19T12:29:59.000Z,Money stop nonsense is the main reason why mos...,1384122103876648967,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,1,0,2,0
3,2021-04-19T12:29:59.000Z,Watching PMs speech right now and honestly it'...,1384122103863996422,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",NaN,2,0,2,0
4,2021-04-19T12:29:59.000Z,@Richard_1942 It's almost as if being part of ...,1384122103822045194,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...
25027,2021-04-19T12:27:56.000Z,guys i miss xinyan so much where is she why do...,1384121584185577480,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0,0,8,0
25028,2021-04-19T12:27:56.000Z,@Banana_da_boi Where and what kind of,1384121584164622342,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",NaN,1,1,0,0
25029,2021-04-19T12:27:56.000Z,@schittock7 Probably on the phone to Niko Kran...,1384121584164622336,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0,0,0,0
25030,2021-04-19T12:27:56.000Z,@HanBigTiddies TAENA 😭😭✋NO TO MOVE ONN,1384121584160448513,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,0,1,0,0


**Drop unwanted columns** ✂️

In [20]:
twitter_tweets = twitter_tweets.drop(columns = ['public_metrics','withheld'])
twitter_tweets

,created_at,text,id,retweet_count,reply_count,like_count,quote_count
0,2021-04-19T12:29:59.000Z,@Sportyfreak2005 ISL played an huge role in de...,1384122103977316355,0,0,1,0
1,2021-04-19T12:29:59.000Z,@O_ssai @AyoOyalowo Was having a conversation ...,1384122103918596098,0,1,7,0
2,2021-04-19T12:29:59.000Z,Money stop nonsense is the main reason why mos...,1384122103876648967,1,0,2,0
3,2021-04-19T12:29:59.000Z,Watching PMs speech right now and honestly it'...,1384122103863996422,2,0,2,0
4,2021-04-19T12:29:59.000Z,@Richard_1942 It's almost as if being part of ...,1384122103822045194,0,0,0,0
...,...,...,...,...,...,...,...
25027,2021-04-19T12:27:56.000Z,guys i miss xinyan so much where is she why do...,1384121584185577480,0,0,8,0
25028,2021-04-19T12:27:56.000Z,@Banana_da_boi Where and what kind of,1384121584164622342,1,1,0,0
25029,2021-04-19T12:27:56.000Z,@schittock7 Probably on the phone to Niko Kran...,1384121584164622336,0,0,0,0
25030,2021-04-19T12:27:56.000Z,@HanBigTiddies TAENA 😭😭✋NO TO MOVE ONN,1384121584160448513,0,1,0,0


**Separate the date and time from created_at column.**

In [21]:
twitter_tweets[['date','time']] = twitter_tweets['created_at'].str.split('T', n=1, expand=True)
twitter_tweets.head()

,created_at,text,id,retweet_count,reply_count,like_count,quote_count,date,time
0,2021-04-19T12:29:59.000Z,@Sportyfreak2005 ISL played an huge role in de...,1384122103977316355,0,0,1,0,2021-04-19,12:29:59.000Z
1,2021-04-19T12:29:59.000Z,@O_ssai @AyoOyalowo Was having a conversation ...,1384122103918596098,0,1,7,0,2021-04-19,12:29:59.000Z
2,2021-04-19T12:29:59.000Z,Money stop nonsense is the main reason why mos...,1384122103876648967,1,0,2,0,2021-04-19,12:29:59.000Z
3,2021-04-19T12:29:59.000Z,Watching PMs speech right now and honestly it'...,1384122103863996422,2,0,2,0,2021-04-19,12:29:59.000Z
4,2021-04-19T12:29:59.000Z,@Richard_1942 It's almost as if being part of ...,1384122103822045194,0,0,0,0,2021-04-19,12:29:59.000Z


In [22]:
twitter_tweets.tail()

,created_at,text,id,retweet_count,reply_count,like_count,quote_count,date,time
25027,2021-04-19T12:27:56.000Z,guys i miss xinyan so much where is she why do...,1384121584185577480,0,0,8,0,2021-04-19,12:27:56.000Z
25028,2021-04-19T12:27:56.000Z,@Banana_da_boi Where and what kind of,1384121584164622342,1,1,0,0,2021-04-19,12:27:56.000Z
25029,2021-04-19T12:27:56.000Z,@schittock7 Probably on the phone to Niko Kran...,1384121584164622336,0,0,0,0,2021-04-19,12:27:56.000Z
25030,2021-04-19T12:27:56.000Z,@HanBigTiddies TAENA 😭😭✋NO TO MOVE ONN,1384121584160448513,0,1,0,0,2021-04-19,12:27:56.000Z
25031,2021-04-19T12:27:56.000Z,I really wish the Die Hard Trilogy for #PS1 ha...,1384121584139468800,0,1,1,0,2021-04-19,12:27:56.000Z


In [23]:
twitter_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25032 entries, 0 to 25031
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   created_at     25032 non-null  object
 1   text           25032 non-null  object
 2   id             25032 non-null  object
 3   retweet_count  25032 non-null  int64 
 4   reply_count    25032 non-null  int64 
 5   like_count     25032 non-null  int64 
 6   quote_count    25032 non-null  int64 
 7   date           25032 non-null  object
 8   time           25032 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB
